The code estimates the **mixing probabilities** (\(\pi_A, \pi_B, \pi_C\)) and the **bias probabilities** (\(p_A, p_B, p_C\)) of three unknown coins using the Expectation-Maximization (EM).
The input: dataset where each row represents a session of 20 coin flips, and the goal is to determine which coin was used in each session and the probability of heads for each coin.

In [ ]:
import numpy as np
from scipy.stats import binom

In [ ]:
def initialize_parameters(n_coins, seed=42):
    np.random.seed(seed)
    p = np.random.rand(n_coins)  # Prob of each coin initialized random
    pi = np.ones(n_coins) / n_coins  # Pr of head random initialized
    return p, pi

In [ ]:
#E-step : computation of the responsabilities matrix for each coin in each session (Pr of each coing of being used in sessions)
def expectation_step(X, D, p, pi): # X = number of sessions #D: number of flliping coins in each session (20)
    N = len(X)
    responsibilities = np.zeros((N, len(p)))
    for j in range(len(p)): #each coin is J
        responsibilities[:, j] = pi[j] * binom.pmf(X, D, p[j]) #Computation of geting head en D flliping using the j coin according to the binomial distribution
      #given the pi(j (Bayes))
    responsibilities /= responsibilities.sum(axis=1, keepdims=True) #Normalizing to sum 1
    return responsibilities

In [ ]:
#M step
def maximization_step(X, D, responsibilities):
  #updating pi and pr with the responsabilities
    pi_new = responsibilities.mean(axis=0)
    p_new = (responsibilities.T @ X) / (responsibilities.T @ np.full(len(X), D))
    return p_new, pi_new

In [ ]:
def expectation_maximization(X, n_coins=3, max_iter=100, tol=1e-6):
    N = len(X)  # number of sessions
    D = 20  # how many times I flip the coin per session

    p, pi = initialize_parameters(n_coins)

    for iteration in range(max_iter):
        responsibilities = expectation_step(X, D, p, pi)
        p_new, pi_new = maximization_step(X, D, responsibilities)

        if np.linalg.norm(p_new - p) < tol and np.linalg.norm(pi_new - pi) < tol:
            break

        pi, p = pi_new, p_new

    return pi, p

In [ ]:
# loading data
def load_data(my_file):
    with open(my_file, 'r') as f:
        data = [[int(bit) for bit in line.split()] for line in f]
    return np.sum(data, axis=1)  # getting head count in each flipping



In [ ]:

my_file= "coin_flips_outcome.txt"
X = load_data(my_file)
pi_est, p_est = expectation_maximization(X)

print("Prob estimation of the mixture (pi_A, pi_B, pi_C):", pi_est) #Prob of each coin of being used
print("Bias estimation (p_A, p_B, p_C):", p_est) #Prob of getting head using according to the coin

Prob estimation of the mixture (pi_A, pi_B, pi_C): [0.30681524 0.1785576  0.51462717]
Bias estimation (p_A, p_B, p_C): [0.23691934 0.93172856 0.61003838]
